# Clojure Macro examples


### ' quote  

In [1]:
(clojure.repl/doc quote)

-------------------------
quote
  (quote form)
Special Form
  Yields the unevaluated form.

  Please see http://clojure.org/special_forms#quote


In [2]:
(= 'a (quote a))

true

In [3]:
'(+ q (w) e)

(+ q (w) e)

In [4]:
(quote (println "foo"))

(println "foo")

In [5]:
(clojure.repl/source quote)

Source not found


In [6]:
(macroexpand ''(1 2 3))

(quote (1 2 3))

### ` syntax quote 

quoting with
- namespace resolution
- possibility to unquote

In [7]:
`x

user/x

In [8]:
(= `x (quote x))

false

In [9]:
'(foo bar)

(foo bar)

In [10]:
`(foo bar)

(user/foo user/bar)

### ~ unquote
"evaluate this item in syntax quoted expression"

In [11]:
(let [x 2]
  `(1 (inc x) 3))

(1 (clojure.core/inc user/x) 3)

In [12]:
(let [x 2]
  `(1 ~(inc x) 3))

(1 3 3)

In [13]:
(let [x 2]
  `(1 (inc ~x) 3))

(1 (clojure.core/inc 2) 3)

In [14]:
`(this ~(symbol (str "i" "s" \- "cool")))

(user/this is-cool)

### ~ and unquote not the same

In [15]:
(clojure.repl/source unquote)

(def unquote)


In [16]:
(clojure.repl/doc unquote)

-------------------------
clojure.core/unquote
  nil


In [17]:
unquote

#object[clojure.lang.Var$Unbound 0x8d934d7 "Unbound: #'clojure.core/unquote"]

### ~ works simple in syntax quoted expression

In [18]:
`(+ 1 (unquote 'x))

(clojure.core/+ 1 (clojure.core/unquote (quote user/x)))

!!! we want just evaluate 'x form ('x -> x)

In [19]:
`(+ 1 ~'x)

(clojure.core/+ 1 x)

### ~@ unquote-splicing

In [20]:
(let [x `(2 3)] 
  `(1 ~x))

(1 (2 3))

In [21]:
(let [x `(2 3)] 
  `(1 ~@x)) 

(1 2 3)

In [22]:
`(1 2 ~(list 3 4))

(1 2 (3 4))

In [23]:
`(1 2 ~@(list 3 4))

(1 2 3 4)

### Gensym
get cool name for var
- gensym function
- var# inside syntax quote

In [24]:
`(let [foo# 1] (+ foo# 2))

(clojure.core/let [foo__5377__auto__ 1] (clojure.core/+ foo__5377__auto__ 2))

In [25]:
`foo#

foo__5381__auto__

In [26]:
(gensym "foo")

foo5386

### Composition 

In [27]:
`[:a ~(+ 1 1) c]

[:a 2 user/c]

In [28]:
`[:a ~(+ 1 1) ~'c]

[:a 2 c]

In [29]:
`[:a ~(+ 1 1) ~`c]

[:a 2 user/c]

In [30]:
`{:a 1 :b '~(+ 1 2)}

{:b (quote 3), :a 1}

In [31]:
`[:a ~(+ 1 1) '~'c]

[:a 2 (quote c)]

In [32]:
`{:a 1 :b '~@(list 1 2)}

{:b (quote 1 2), :a 1}

In [33]:
`(1 `(2 3) 4)

(1 (clojure.core/seq (clojure.core/concat (clojure.core/list 2) (clojure.core/list 3))) 4)

In [34]:
`(list 1 `(2 ~(- 9 6)) 4) ; twice syntax quoted and unquote

(clojure.core/list 1 (clojure.core/seq (clojure.core/concat (clojure.core/list 2) (clojure.core/list (clojure.core/- 9 6)))) 4)

In [35]:
`(list 1 `(2 ~~(- 9 6)) 4) ; twice syntax quoted and twice unquoted

(clojure.core/list 1 (clojure.core/seq (clojure.core/concat (clojure.core/list 2) (clojure.core/list 3))) 4)

In [36]:
(eval `(list 1 `(2 ~~(- 9 6)) 4))

(1 (2 3) 4)

In [37]:
(= `'~(+ 2 3) `(quote ~(+ 2 3)) (list 'quote 5) '(quote 5))

true

## Macros and functions

### Macros not evaluate arguments

In [38]:
(defmacro simple-macro [a] (do (println "arg: " a) a))
(defn simple-fun [a] (do (println "arg: "a) a))

#'user/simple-fun

In [39]:
(simple-macro "test")

arg:  test


"test"

In [40]:
(simple-fun "test")

arg:  test


"test"

### But

In [41]:
(simple-macro (+ 1 2))

arg:  (+ 1 2)


3

In [42]:
(simple-fun (+ 1 2))

arg:  3


3

### Functions evaluate arguments first and can't see *(+ 1 2)* 
* Functions evaluate arguments ~~result~~
* Macros   evaluate ~~arguments~~ result

```
(= (macroexpand-1 '(foo-m arg1 arg2 ...))
   (foo-f 'arg1 'arg2 '...))
   
(= (foo-m arg1 arg2 ...)
   (eval (foo-f 'arg1 'arg2 '...)))
```

In [43]:
(defmacro m-dec2 [x]
    `(- ~x 2))

#'user/m-dec2

In [44]:
(m-dec2 5)

3

In [45]:
(macroexpand-1 '(m-dec2 5))

(clojure.core/- 5 2)

In [46]:
(defn f-dec2 [x] (- x 2))

#'user/f-dec2

In [47]:
(f-dec2 5)

3

In [48]:
(let [y 7]
  (= (+ (m-dec2 y) 6)
     (+ (f-dec2 y) 6)))

true

In [49]:
(defmacro represent1 [x] `'~x)
(represent1 (+ 3 54))

(+ 3 54)

In [50]:
(defmacro represent2 [x] `(quote ~x))
(represent2 (+ 3 54))

(+ 3 54)

In [51]:
(defmacro represent-and-eval [x] `(str '~x " >> " ~x))
(represent-and-eval (+ 3 54))

"(+ 3 54) >> 57"

### Inside defmacro
**&form**,
**&env**

In [52]:
(defmacro show-env [] (println &env))
(show-env)

nil


In [53]:
(let [band "zeppelin" city "london"] (show-env))

{band #object[clojure.lang.Compiler$LocalBinding 0x5fb96ae6 clojure.lang.Compiler$LocalBinding@5fb96ae6], city #object[clojure.lang.Compiler$LocalBinding 0x74b88275 clojure.lang.Compiler$LocalBinding@74b88275]}


In [54]:
(defmacro show-env-info []
    (println (keys &env)) ; symbols
    (println (map class (keys &env)))
    (println (map namespace (keys &env)))  ; don't have namespace
    `(println ~@(keys &env))) ; have values
(let [band "zeppelin" city "london"] (show-env-info)) 

(band city)
(clojure.lang.Symbol clojure.lang.Symbol)
(nil nil)
zeppelin london


In [55]:
(defmacro show-form [a b] (println &form)) 
(show-form 50 undefined-var)

(show-form 50 undefined-var)


In [56]:
(defmacro show-classes [a b] (println (map (juxt identity class) &form)))
(show-classes 10 "10")

([show-classes clojure.lang.Symbol] [10 java.lang.Long] [10 java.lang.String])


In [57]:
(defmacro show-meta [a b] (println (meta &form)))
(show-meta 10 "10")

{:line 2, :column 1}


In [58]:
(meta (show-meta 10 "10"))
(meta 
 (show-meta 10 "10"))

{:line 1, :column 7}
{:line 3, :column 2}


**Links**
- https://clojuredocs.org/
- https://8thlight.com/blog/colin-jones/2012/05/22/quoting-without-confusion.html
- https://blog.klipse.tech/clojure/2016/05/01/macro-tutorial-1.html
- https://clojure.org/reference/reader
- https://clojure.org/reference/macros
- http://blog.jayfields.com/2011/02/clojure-and.html